In [167]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,LabelEncoder




In [168]:
data_main = pd.read_csv(r"C:\Users\Arpit\Desktop\House_Prediction_Project\Bengaluru_House_Data.csv")



In [169]:
data_main=data_main.drop(["society","area_type","availability","balcony"], axis = 1)

In [170]:
data_main["location"]=data_main["location"].fillna("Sarjapur  Road")
data_main["size"]=data_main["size"].fillna("2 BHK")
data_main["bath"]=data_main["bath"].fillna(data_main["bath"].median())



In [171]:
data_main.sample(15)

,location,size,total_sqft,bath,price
8840,Chikka Tirupathi,4 Bedroom,4500,5.0,150.00
9182,Yeshwanthpur,2 BHK,1165,2.0,64.08
1292,Channasandra,2 BHK,1009,2.0,38.34
3279,Hennur Road,3 BHK,2350,3.0,174.00
10999,Kaggadasapura,5 Bedroom,2400,4.0,130.00
280,Gollarapalya Hosahalli,2 BHK,861,2.0,34.50
11909,Kanakpura Road,3 BHK,1300,2.0,69.00
10001,Koramangala,2 BHK,1320,2.0,160.00
12710,Sarjapur,2 BHK,1095,2.0,45.00
7805,Punappa Layout,3 BHK,1310,2.0,55.00


In [172]:
data_main["bhk"]=data_main["size"].str.split().str.get(0).astype(int)

In [173]:
data_main.sample(12)

,location,size,total_sqft,bath,price,bhk
7162,Lakshmipura,2 Bedroom,1225,2.0,65.00,2
3294,Haralur Road,2 BHK,1464,2.0,56.00,2
12498,Vittasandra,2 BHK,1404,2.0,67.50,2
6947,Shampura,3 BHK,1700,3.0,75.00,3
8705,Rayasandra,4 BHK,1708,3.0,76.00,4
12344,Siddapura,2 BHK,1150,2.0,90.00,2
6191,Bommasandra,2 BHK,1020,2.0,40.89,2
5879,Kanakpura Road,3 BHK,1665,3.0,86.58,3
4553,Sarjapur Road,2 BHK,1200,2.0,93.00,2
12754,Gopalkrishna Nagar,3 BHK,1570,2.0,95.00,3


In [174]:
data_main=data_main.drop("size",axis=1)

In [175]:
data_main

,location,total_sqft,bath,price,bhk
0,Electronic City Phase II,1056,2.0,39.07,2
1,Chikka Tirupathi,2600,5.0,120.00,4
2,Uttarahalli,1440,2.0,62.00,3
3,Lingadheeranahalli,1521,3.0,95.00,3
4,Kothanur,1200,2.0,51.00,2
...,...,...,...,...,...
13315,Whitefield,3453,4.0,231.00,5
13316,Richards Town,3600,5.0,400.00,4
13317,Raja Rajeshwari Nagar,1141,2.0,60.00,2
13318,Padmanabhanagar,4689,4.0,488.00,4


In [176]:
data_main["location"].value_counts()

location
Whitefield                        540
Sarjapur  Road                    400
Electronic City                   302
Kanakpura Road                    273
Thanisandra                       234
                                 ... 
Bapuji Layout                       1
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
singapura paradise                  1
Abshot Layout                       1
Name: count, Length: 1305, dtype: int64

In [177]:
data_main["location"]=data_main["location"].apply(lambda x:x.strip())

In [178]:
location_count = data_main["location"].value_counts()
location_count

location
Whitefield                        541
Sarjapur  Road                    400
Electronic City                   304
Kanakpura Road                    273
Thanisandra                       237
                                 ... 
Bapuji Layout                       1
1st Stage Radha Krishna Layout      1
BEML Layout 5th stage               1
singapura paradise                  1
Abshot Layout                       1
Name: count, Length: 1294, dtype: int64

In [179]:
location_10 = location_count[location_count<=11]
location_10

location
Nehru Nagar                       11
Banjara Layout                    11
Tindlu                            11
Kodigehalli                       11
Marsur                            11
                                  ..
Bapuji Layout                      1
1st Stage Radha Krishna Layout     1
BEML Layout 5th stage              1
singapura paradise                 1
Abshot Layout                      1
Name: count, Length: 1064, dtype: int64

In [180]:
data_main["location"]= data_main["location"].apply(lambda x: "others" if x in location_10 else x)
data_main["location"].sample(15)


8416                others
11555         Kasavanhalli
8098                others
12960               others
7272             Yelahanka
4111     Somasundara Palya
5048         Akshaya Nagar
2022                others
13253      Old Madras Road
13298        Kenchenahalli
11590    Bannerghatta Road
8616                others
12454          Mahadevpura
2860           Koramangala
8811         Kaggadasapura
Name: location, dtype: object

In [181]:
onehot = OneHotEncoder()

ohe = OneHotEncoder(drop='first',sparse_output=False,dtype=np.int32)

In [182]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# 1. Reshape the Series to 2D (n_samples, 1)
locations_reshaped = data_main['location'].values.reshape(-1, 1)

# 2. Initialize the encoder
onehot = OneHotEncoder(sparse_output=False)  # sparse=False to get a dense array

# 3. Fit and transform
location_encoded = onehot.fit_transform(locations_reshaped)

# 4. Convert to DataFrame with proper column names
location_encoded_df = pd.DataFrame(location_encoded, columns=onehot.get_feature_names_out(['location']))

# 5. Merge back to original DataFrame
data_main = pd.concat([data_main, location_encoded_df], axis=1)

# 6. Drop the original 'location' column if needed
data_main = data_main.drop('location', axis=1)


In [183]:
data_main

,total_sqft,bath,price,bhk,location_1st Block Jayanagar,location_1st Phase JP Nagar,location_2nd Stage Nagarbhavi,location_5th Block Hbr Layout,location_5th Phase JP Nagar,location_6th Phase JP Nagar,location_7th Phase JP Nagar,location_8th Phase JP Nagar,location_9th Phase JP Nagar,location_AECS Layout,location_Abbigere,location_Akshaya Nagar,location_Ambalipura,location_Ambedkar Nagar,location_Amruthahalli,location_Anandapura,location_Ananth Nagar,location_Anekal,location_Anjanapura,location_Ardendale,location_Arekere,location_Attibele,location_BEML Layout,location_BTM 2nd Stage,location_BTM Layout,location_Babusapalaya,location_Badavala Nagar,location_Balagere,location_Banashankari,location_Banashankari Stage II,location_Banashankari Stage III,location_Banashankari Stage V,location_Banashankari Stage VI,location_Banaswadi,location_Bannerghatta,location_Bannerghatta Road,...,location_Sahakara Nagar,location_Sanjay nagar,location_Sarakki Nagar,location_Sarjapur,location_Sarjapur Road,location_Sarjapura - Attibele Road,location_Sector 2 HSR Layout,location_Sector 7 HSR Layout,location_Seegehalli,location_Shampura,location_Shivaji Nagar,location_Singasandra,location_Somasundara Palya,location_Sompura,location_Sonnenahalli,location_Subramanyapura,location_Sultan Palaya,location_TC Palaya,location_Talaghattapura,location_Thanisandra,location_Thigalarapalya,location_Thubarahalli,location_Tumkur Road,location_Ulsoor,location_Uttarahalli,location_Varthur,location_Varthur Road,location_Vasanthapura,location_Vidyaranyapura,location_Vijayanagar,location_Vishveshwarya Layout,location_Vishwapriya Layout,location_Vittasandra,location_Whitefield,location_Yelachenahalli,location_Yelahanka,location_Yelahanka New Town,location_Yelenahalli,location_Yeshwanthpur,location_others
0,1056,2.0,39.07,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2600,5.0,120.00,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1440,2.0,62.00,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1521,3.0,95.00,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1200,2.0,51.00,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13315,3453,4.0,231.00,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [184]:
data_main.shape

(13320, 235)